# 3) Analyze and evaluate optimization output - TEST PROTOCOLS

This final notebook uses the `runs.pkl` file created in notebook 2 and it analyzes:

- the distance between different strategies in the feature space - test protocols


In [ ]:
import pickle
import pandas as pd
import seaborn as sns
import sys
import shutil

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import matplotlib.pyplot as plt
from scipy.spatial import distance
import MEAutility as mu
import json
import time
import numpy as np
from pathlib import Path
from pprint import pprint

from scipy.stats import kruskal, mannwhitneyu, wilcoxon

import multimodalfitting as mf

%matplotlib notebook

In [ ]:
save_fig = False
figure_folder = Path(".") / "figures_hay_ais"

if save_fig:
    figure_folder.mkdir(exist_ok=True)

In [ ]:
base_dir = Path("../..")

In [ ]:
colors_dict = {"soma": "C0",
               "all": "C1",
               "sections": "C2",
               "single": "C3"}
feature_sets = {"soma": "soma",
                "all": "extra",
                "sections": "extra",
                "single": "extra"}
figsize = (10, 7)

## Load GT params and optimization output

In [ ]:
# general
model_name = "hay_ais"
probe_type = "planar" # linear 

cell_models_folder = base_dir / "cell_models"
model_folder = cell_models_folder / model_name
probe_file = model_folder / "fitting" / "efeatures" / "probe_BPO.json"

In [ ]:
# change this with folder containing your pkl file
results_date = '220120'  # '211124' '220111' # 
result_folder = base_dir / "results" / results_date

In [ ]:
cell = mf.create_ground_truth_model(model_name=model_name,
                                    release=False)
cell_release = mf.create_ground_truth_model(model_name=model_name,
                                            release=True)

probe = mf.define_electrode(probe_file=probe_file)

param_names = [param.name for param in cell.params.values() if not param.frozen]

params_release = {}
for param in cell_release.params_by_names(param_names):
    params_release[param.name] = param.value

In [ ]:
protocol_for_eap = "firepattern_200"

In [ ]:
pkl_file_name = "runs.pkl"

In [ ]:
data = pickle.load(open(result_folder / pkl_file_name, 'rb'))
df_optimization = pd.DataFrame(data)
df_model = df_optimization.query(f"model == '{model_name}'")

In [ ]:
# set strategy column
df_model.loc[:, "strategy"] = df_model["extra_strategy"].values.copy()
df_model.loc[df_model["feature_set"] == "soma", "strategy"] = "soma"

In [ ]:
fig, ax = plt.subplots()
min_evals = 3000
color_strategy = {"all": "C1", "sections": "C2", "single": "C3"}

keep_idxs = []
for idx, row in df_model.iterrows():
    if max(row["nevals"]) > min_evals:
        keep_idxs.append(idx)
        ax.plot(row["nevals"], 
                row["logbook"].select("min"),
                color=colors_dict[row["strategy"]],
                ls='-', 
                lw=0.8,
                alpha=0.75)
    else:
        ax.plot(row["nevals"], 
                row["logbook"].select("min"),
                color=colors_dict[row["strategy"]],
                ls='--', 
                lw=0.5,
                alpha=0.75)

ax.set_title("Min fitness")
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_xlabel("Neval")
ax.set_ylabel("Min fitness")
ax.set_yscale('log')

# Load protocols and original features

In [ ]:
extra_kwargs = mf.utils.get_extra_kwargs()
extra_kwargs

In [ ]:
protocols_used_for_opt = ["IV_-20", "IV_-100", "IDrest_150", "IDrest_250", "IDrest_300",
                          "APWaveform_260"]

In [ ]:
eva_extra = mf.create_evaluator(
    model_name=model_name,
    feature_set="extra",
    extra_strategy="all",
    protocols_with_lfp="IDrest_300",
    all_protocols=True,
    exclude_protocols=protocols_used_for_opt,
    **extra_kwargs
)

In [ ]:
print(f"All test features --> num features {len(eva_extra.fitness_calculator.objectives)}")

# Compute release responses

In [ ]:
responses_release = eva_extra.run_protocols(eva_extra.fitness_protocols.values(), param_values=params_release)

In [ ]:
features_release = {}
for obj in eva_extra.fitness_calculator.objectives:
    features_release[obj.features[0].name] = {}
    if len(obj.features) == 1:
        feat = obj.features[0]
        feat_value = feat.calculate_feature(responses_release)
        feat_score = feat.calculate_score(responses_release)
        features_release[feat.name]["value"] = feat_value
    else:
        print(f"More than one feature for objective: {obj.name}")

In [ ]:
pprint(list(responses_release.keys()))

In [ ]:
fig_gt_intra = mf.plot_responses(responses_release, color="k", return_fig=True)

In [ ]:
if save_fig:
    fig_gt_intra.savefig(figure_folder / "gt_intra.pdf")
    fig_gt_extra.savefig(figure_folder / "gt_extra.pdf")

# Compute and plot best responses

In [ ]:
max_feature_value = 50
opt_results = {}

In [ ]:
for strategy in np.unique(df_model.strategy):
    opt_results[strategy] = {}
    opt_df = df_model.query(f"strategy == '{strategy}'")
    best_idx = np.argmin(opt_df.best_fitness)
    params_sample = opt_df.iloc[best_idx]
    params_dict = {k: v for k, v in zip(param_names, params_sample.best_params)}
    opt_results[strategy]["best_fitness"] = params_sample.best_fitness
    opt_results[strategy]["best_params"] = params_dict
    print(f"{strategy} --  best fitness: {params_sample.best_fitness}")

In [ ]:
for strategy in np.unique(df_model.strategy):
    print(f"Simulating best '{strategy}'")
    responses = eva_extra.run_protocols(eva_extra.fitness_protocols.values(), 
                                        param_values=opt_results[strategy]["best_params"])
    opt_results[strategy]["responses"] = responses

In [ ]:
for strategy in np.unique(df_model.strategy):
    responses = opt_results[strategy]["responses"]
    features_best = {}
    for obj in eva_extra.fitness_calculator.objectives:
        feat = obj.features[0]
        features_best[feat.name] = {}
        if len(obj.features) == 1:
            feat_value = obj.features[0].calculate_feature(responses)
            if feat_value is None:
                feat_value = max_feature_value
            features_best[feat.name]["value"] = feat_value
            feat_score = np.abs(features_release[feat.name]["value"] - feat_value) / feat.exp_std
            features_best[feat.name]["score"] = feat_score
        else:
            print(f"More than one feature for objective: {obj.name}")
    opt_results[strategy]["features"] = features_best

In [ ]:
for strategy in np.unique(df_model.strategy):
    responses_to_plot = [responses_release, opt_results[strategy]["responses"]]
    colors = ["k", colors_dict[strategy]]
    labels = ["GT", strategy.upper()]
    fig_extra_intra_single = mf.plot_multiple_responses(responses_to_plot, 
                                                        colors=colors, return_fig=True, 
                                                        labels=labels)

In [ ]:
if save_fig:
    fig_extra_intra.savefig(figure_folder / "extra_intra.pdf")
    fig_extra_extra.savefig(figure_folder / "extra_extra.pdf")
    fig_extra_single_intra.savefig(figure_folder / "single_intra.pdf")
    fig_extra_single.savefig(figure_folder / "single_extra.pdf")

## Compare best-fitted models

### Compare features

In [ ]:
feature_name_array = []
feature_set_array = []
feature_score_array = []
feature_type_array = []
protocol_type_array = []

for strategy, strategy_dict in opt_results.items():
    feats = strategy_dict["features"]
    for feat_name, feat_dict in feats.items():
        feature_set_array.append(strategy)
        feature_name_array.append(feat_name)
        if "MEA" not in feat_name:
            feature_type_array.append("intra")
        else:
            feature_type_array.append("extra")
        feature_score_array.append(feat_dict["score"])
        protocol_type = feat_name.split(".")[0].split("_")[0]
        protocol_type_array.append(protocol_type)
        
df_feats = pd.DataFrame({"feature_set": feature_set_array, "feat_name": feature_name_array,
                         "feat_score": feature_score_array, "protocol_type": protocol_type_array})

In [ ]:
df_feats

In [ ]:
fig_feat_intra, ax = plt.subplots(figsize=(10, 7))

sns.boxplot(data=df_feats, y="feature_set", x="feat_score",# hue="protocol_type", 
            ax=ax)
# g = sns.swarmplot(data=df_feats, y="feature_set", x="feat_score", ax=ax)
ax.set_ylabel("Feature scores", fontsize=12)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_title("Intracellular features", fontsize=15)
ax.set_xlim([0, 20])

In [ ]:
for protocol_type in np.unique(df_feats.protocol_type):
    fig, ax = plt.subplots(figsize=(10, 7))
    sns.barplot(data=df_feats.query(f"protocol_type == '{protocol_type}'"), 
                y="feature_set", x="feat_score",# hue="protocol_type", 
                ax=ax)
    ax.set_title(protocol_type, fontsize=15)
    ax.set_ylabel("Feature scores", fontsize=12)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
#     ax.set_title("Intracellular features", fontsize=15)
#     ax.set_xlim([0, 20])

In [ ]:
if save_fig:
    fig_feat_intra.savefig(figure_folder / "feat_intra.pdf")
    fig_feat_extra.savefig(figure_folder / "feat_extra.pdf")